<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import special

Create a `ContinuousDistribution` from `scipy.stats.loguniform`.


In [ ]:
LogUniform = stats.make_distribution(stats.loguniform)
X = LogUniform(a=1.0, b=3.0)
np.isclose((X + 0.25).median(), stats.loguniform.ppf(0.5, 1, 3, loc=0.25))

np.True_

In [ ]:
X.plot()
sample = X.sample(10000, rng=np.random.default_rng())
plt.hist(sample, density=True, bins=30)
plt.legend(('pdf', 'histogram'))
plt.show()

Create a custom distribution.


In [ ]:
class MyLogUniform:
    @property
    def __make_distribution_version__(self):
        return "1.16.0"

    @property
    def parameters(self):
        return {'a': {'endpoints': (0, np.inf),
                      'inclusive': (False, False)},
                'b': {'endpoints': ('a', np.inf),
                      'inclusive': (False, False)}}

    @property
    def support(self):
        return {'endpoints': ('a', 'b'), 'inclusive': (True, True)}

    def pdf(self, x, a, b):
        return 1 / (x * (np.log(b)- np.log(a)))

MyLogUniform = stats.make_distribution(MyLogUniform())
Y = MyLogUniform(a=1.0, b=3.0)
np.isclose(Y.cdf(2.), X.cdf(2.))

np.True_

Create a custom distribution with variable support.


In [ ]:
class MyUniformCube:
    @property
    def __make_distribution_version__(self):
        return "1.16.0"

    @property
    def parameters(self):
        return {"a": (-np.inf, np.inf),
                "b": {'endpoints':('a', np.inf), 'inclusive':(True, False)}}

    @property
    def support(self):
        def left(*, a, b):
            return a**3

        def right(*, a, b):
            return b**3
        return (left, right)

    def pdf(self, x, *, a, b):
        return 1 / (3*(b - a)*np.cbrt(x)**2)

    def cdf(self, x, *, a, b):
        return (np.cbrt(x) - a) / (b - a)

MyUniformCube = stats.make_distribution(MyUniformCube())
X = MyUniformCube(a=-2, b=2)
Y = stats.Uniform(a=-2, b=2)**3
X.support()

(-8.0, 8.0)

In [ ]:
np.isclose(X.cdf(2.1), Y.cdf(2.1))

np.True_

Create a custom distribution with multiple parameterizations. Here we create a
custom version of the beta distribution that has an alternative parameterization
in terms of the mean ``mu`` and a dispersion parameter ``nu``.


In [ ]:
class MyBeta:
    @property
    def __make_distribution_version__(self):
        return "1.16.0"

    @property
    def parameters(self):
        return ({"a": (0, np.inf), "b": (0, np.inf)},
                {"mu": (0, 1), "nu": (0, np.inf)})

    def process_parameters(self, a=None, b=None, mu=None, nu=None):
        if a is not None and b is not None:
            nu = a + b
            mu = a / nu
        else:
            a = mu * nu
            b = nu - a
        return dict(a=a, b=b, mu=mu, nu=nu)

    @property
    def support(self):
        return {'endpoints': (0, 1)}

    def pdf(self, x, a, b, mu, nu):
        return special._ufuncs._beta_pdf(x, a, b)

    def cdf(self, x, a, b, mu, nu):
        return special.betainc(a, b, x)

MyBeta = stats.make_distribution(MyBeta())
X = MyBeta(a=2.0, b=2.0)
Y = MyBeta(mu=0.5, nu=4.0)
np.isclose(X.pdf(0.3), Y.pdf(0.3))

np.True_